# Introduction

**Main Topic**

This notebook is for **generating coco format json files** using **images** and **image_labels.csv**

Becasue I don't know how to approch this competition well, I'm trying to make Object Detection Model using images and videos first.

So for the first step, I converted bbox informations to coco json format.

Please tell me if there is any bug or improvement.

after this nb you can refer [Detecting Helmet using MMdetection w/o internet](https://www.kaggle.com/jinssaa/detecting-helmet-using-mmdetection-w-o-internet) to building 

`Helmet` detection model

**References**

I just customized this code from my old files, so please refer to me if there is any references code made by below style 

# Set up environment

In [ ]:
#-*- coding: utf-8 -*-

import json
import os
import cv2
import string
import random
import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
image_labels = pd.read_csv('../input/nfl-impact-detection/image_labels.csv')

In [ ]:
image_labels.head()

In [ ]:
def gen_classes(CLASSES):
    classes = list()      
    
    for i, CLASS in enumerate(CLASSES):
        single_class = {} 
        single_class['id'] = i + 1
        single_class['name'] = CLASS
        classes.append(single_class)
    return classes

In [ ]:
CLASSES = list(image_labels['label'].unique())
classes = gen_classes(CLASSES)
classes

I'm not sure but we might customize this classse to kind of {helmet, helmet-impact, shoulder-impact} etc, 

or predicting helmet can make some rule-based inference. I will just set `classes` only `Helmet`

In [ ]:
classes = classes[0]

In [ ]:
def gen_objs(df, validation_ratio = 0.2, debug = False):
    
    if debug:
        
        df = df[:int(len(df)*0.05)]
        
    
    img_lists = list(df['image'].unique()) ## gen unique image lists
    imgs = list()
    train_objs = list()
    valid_objs = list() 
    
    ## gen images information
    for i in tqdm(range(len(img_lists))):
        '''
        
        I just notice that all images were preprocessed image size 720x1280
        If you want to check real image size, use this code below
        
        img = cv2.imread(os.path.join(data_path, img_lists[i]))
        
        single_img_obj = {}
        single_img_obj['file_name'] = img_lists[i]
        single_img_obj['height'] = img.shape[0]
        single_img_obj['width'] = img.shape[1]
        single_img_obj['id'] = i + 1
        '''
        
        single_img_obj = {}
        single_img_obj['file_name'] = img_lists[i]
        single_img_obj['height'] = 720 
        single_img_obj['width'] = 1280
        single_img_obj['id'] = i + 1        
        
        imgs.append(single_img_obj)
        
        
    ## gen train & val objs information        
    train_df, valid_df = train_test_split(df, test_size = validation_ratio, random_state=42)
    train_df = train_df.reset_index()
    valid_df = valid_df.reset_index()
    
    ## gen train objs information    
    for j in tqdm(range(len(train_df))):
        single_obj = {}
        single_obj['id'] = j + 1
        single_obj['image_id'] = img_lists.index(df['image'][j]) + 1
        single_obj['category_id'] = 1 # If you wanna set multi-class, set CLASSES.index(df['label'][j]) + 1
        single_obj['area'] = float(df['width'][j]*df['height'][j])
        single_obj['bbox'] = [int(df['left'][j]), int(df['top'][j]), int(df['width'][j]), int(df['height'][j])] ## [min_x, min_y, width, height]
        single_obj['iscrowd'] = 0        
        
        train_objs.append(single_obj)
        
    ## gen valid objs information
    for k in tqdm(range(len(valid_df))):
        single_obj = {}
        single_obj['id'] = k + 1
        single_obj['image_id'] = img_lists.index(df['image'][k]) + 1
        single_obj['category_id'] = 1 # If you wanna set multi-class, set CLASSES.index(df['label'][j]) + 1
        single_obj['area'] = float(df['width'][k]*df['height'][k])
        single_obj['bbox'] = [int(df['left'][k]), int(df['top'][k]), int(df['width'][k]), int(df['height'][k])]
        single_obj['iscrowd'] = 0        
        
        valid_objs.append(single_obj)        
    
    print(f'images: {len(imgs)}, train objs: {len(train_objs)}, valid objs: {len(valid_objs)}')
    
    return imgs, train_objs, valid_objs

In [ ]:
imgs, train_objs, valid_objs = gen_objs(image_labels)

In [ ]:
imgs[:5]

In [ ]:
train_objs[:5]

In [ ]:
def gen_coco(outpath, classes, objs, imgs, train=True):
    if train:
        data_dict = {}
        data_dict['images'] = []
        data_dict['annotations'] = []
        data_dict['categories'] = []
        data_dict['images'].extend(imgs)
        data_dict['annotations'].extend(objs)
        data_dict['categories'].extend(classes)
        
    else:
        data_dict = {}
        data_dict['images'] = []
        data_dict['categories'] = []
        
        data_dict['images'].extend(imgs)
        data_dict['categories'].extend(classes)   

    with open(outpath, 'w') as f_out:
        json.dump(data_dict, f_out)
    print(f'file generate at {outpath}')

In [ ]:
gen_coco('train.json', classes, train_objs, imgs, train=True)
gen_coco('valid.json', classes, valid_objs, imgs, train=True)